# 4-D quantum spacetime with spin network states

## Problem description
problem description goes here
$$ 
\in
A = \bra{W}\ket{\phi}
$$

## Setup
Load code from Cirq and our LQG directory

In [27]:
import cirq
import numpy as np
import os
import json
os.chdir('LQG')
import tetrahedra_num as tn
import tetrahedra_circuits as tc
import postprocess as post
import imp
imp.reload(tn)
imp.reload(tc)
imp.reload(post)
os.chdir('..')

## Hardware layout
This project was executed on the Google Sycamore processor (Weber specifically), whose 2D-qubit layout is pictured below.

In [16]:
print(cirq.google.devices.Sycamore)

                                             (0, 5)───(0, 6)
                                             │        │
                                             │        │
                                    (1, 4)───(1, 5)───(1, 6)───(1, 7)
                                    │        │        │        │
                                    │        │        │        │
                           (2, 3)───(2, 4)───(2, 5)───(2, 6)───(2, 7)───(2, 8)
                           │        │        │        │        │        │
                           │        │        │        │        │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)───(3, 6)───(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │        │        │        │        │
                  │        │        │        │        │        │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)───(4, 6)───(4, 7)───(4, 8)───(4, 9)
         │        │        │        │        │        │   

# Experiment 1
The first submission contained ~3.5 million circuit samples.

First, we experimentally determined the fidelity of preparing the states $\ket{0_L}$ and $\ket{1_L}$.

Second, we experimentally calculated the probability of ending in the $\ket{0}$ state for a monopole network, dipole network, and pentagram network, which is also called a 4-simplice and is the boundary state for a 4-d spacetime.

Lastly, we glued two 4-simplices together at two tetrahedra and calculated the probability of ending in $\ket{0}$. This experiment is more speculative and has not been performed before.

## Load data

In [20]:
results_1 = json.load(open(tc.project_name + '/lqg_ha_cr_measurements_submission_1.json'))

## State preparation fidelity

In [22]:
zero_fidelity = results_1['zero_fidelity']
one_fidelity = results_1['one_fidelity']

### Fidelity of the $\ket{0_L}$ state

In [29]:
for key,value in zero_fidelity.items():
    bitstr = tc.binformat(4).format(int(key))
    print(bitstr+': ', value)
postsel_zero_fidelity, postsel_zero_count = post.post_select_even(zero_fidelity,4)
for key,value in postsel_zero_fidelity.items():
    bitstr = tc.binformat(4).format(int(key))
    print('{}: {} P({})'.format(bitstr,value,value/postsel_zero_count))

0110:  242
1010:  220
0101:  218
1001:  208
1101:  4
0100:  19
0010:  17
1011:  8
0001:  19
1000:  25
0111:  10
1110:  5
0000:  2
0011:  2
1100:  1


ValueError: Unknown format code 'b' for object of type 'str'